In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import shutil
import time, gc, random, math

import torch
from torch.utils.data import DataLoader, Dataset

import transformers
from transformers import TrainingArguments, Trainer, DataCollatorForWholeWordMask
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel

from torch import nn
from torch.optim import Adam, SGD, AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
df = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
df_context = pd.read_csv("/kaggle/input/cpc-codes/titles.csv")

In [ ]:
df.head()

In [ ]:
df_context.head()

In [ ]:
# merge dataframe
df = df.merge(df_context, how='left', 
              left_on='context', right_on='code')
df.columns

In [ ]:
df = df[['id', 'anchor', 'target', 'context',
         'title', 'score']]

In [ ]:
df.head()

In [ ]:
test_df = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")
test_df = test_df.merge(df_context, how='left',
                       left_on='context', right_on='code')
test_df = test_df[['id', 'anchor', 'target', 'context', 'title']]

test_df.head()

In [ ]:
# load model
modelPath = "/kaggle/input/debertav3small"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(modelPath)

In [ ]:
df['input'] = df['target'] + tokenizer.sep_token + df['title'].apply(str.lower)

In [ ]:
test_df['input'] = test_df['target'] + tokenizer.sep_token + test_df['title'].apply(str.lower)

In [ ]:
# train vaild split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=123)

In [ ]:
df.head(2)

In [ ]:
class TrainDatset(Dataset):
    def __init__(self,df):
        self.input = df['input'].values.astype(str)
        self.anchor = df['anchor'].values.astype(str)
        self.label = df['score'].values
        
    def __len__(self):
        return len(self.input)
    
    def __getitem__(self, item):
        inputs = self.input[item]
        anchor = self.anchor[item]
        label = self.label[item]
        
        model_inputs = tokenizer(inputs, anchor,
                                max_length=100,
                                padding='max_length',
                                truncation=True)
        
        return {**model_inputs, 'label':torch.as_tensor(label, dtype=torch.float)}

In [ ]:
class TestDatset(Dataset):
    def __init__(self,df):
        self.input = df['input'].values.astype(str)
        self.anchor = df['anchor'].values.astype(str)
        
        
    def __len__(self):
        return len(self.input)
    
    def __getitem__(self, item):
        inputs = self.input[item]
        anchor = self.anchor[item]
        
        model_inputs = tokenizer(inputs, anchor,
                                max_length=100,
                                padding='max_length',
                                truncation=True)
        
        return {**model_inputs}

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    
    return {
        'pearson':np.corrcoef(predictions, labels)[0][1]
    }

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(modelPath, num_labels=1)

In [ ]:
metric_name = 'pearson'
batch_size = 128

args = TrainingArguments(
        'model',
        evaluation_strategy='epoch',
        save_strategy='epoch',
        learning_rate = 2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size*2,
        num_train_epochs=4,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model=metric_name,
        save_total_limit=1
        )

In [ ]:
train_dataset = TrainDatset(train_df)
valid_dataset = TrainDatset(val_df)

In [ ]:
trainer = Trainer(
            model, 
            args,
            train_dataset=train_dataset,
            eval_dataset=valid_dataset,
            tokenizer= tokenizer,
            compute_metrics=compute_metrics
        )

In [ ]:
trainer.train()

In [ ]:
test_dataset = TestDatset(test_df)

In [ ]:
outputs = trainer.predict(test_dataset).predictions.astype('float')
outputs[:5]

In [ ]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': test_df['id'],
    'score': outputs.flatten()
})

submission.to_csv('submission.csv', index=False)